In [7]:
import os
import cv2 # OpenCV
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm.auto import tqdm # Для індикатора прогресу
import glob # Для пошуку файлів

# ---------------------------------
# 1. КОНФІГУРАЦІЯ
# ---------------------------------
print("--- Етап 1: Конфігурація ---")

# --- ШЛЯХИ ДО МОДЕЛЕЙ ---
# Переконайтеся, що це шлях до ПЕРЕНАВЧЕНОЇ v3 моделі
CLASSIFIER_MODEL_PATH = './models/digit_classifier_best.keras'
YOLO_MODEL_PATH = './runs/detect/train9/weights/best.pt' # (шлях до вашого YOLO)

# --- ШЛЯХИ ДО ВАЛІДАЦІЙНИХ ДАНИХ ---
VALID_IMG_DIR = './learning/data_number/valid/images/'
VALID_LBL_DIR = './learning/data_number/valid/labels/'

# --- Параметри ---
CLASSIFIER_IMG_SIZE = (64, 64)
# Переконайтеся, що список класів правильний (з GARBAGE або без,
# залежно від того, як ви навчили v3)
CLASS_NAMES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
CONFIDENCE_THRESHOLD = 0.5 # Поріг впевненості для YOLO

# ---------------------------------
# 2. ЗАВАНТАЖЕННЯ ФУНКЦІЙ І МОДЕЛЕЙ
# ---------------------------------
print("--- Етап 2: Завантаження моделей та функцій ---")

# --- Функція передобробки (ФІНАЛЬНА ВЕРСІЯ) ---
def preprocess_for_classifier(image_crop_gray, target_size=(64, 64)):
    h, w = image_crop_gray.shape[:2]
    if h == 0 or w == 0:
        return np.zeros((1, target_size[0], target_size[1], 1), dtype=np.float32)

    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)
    if new_h <= 0 or new_w <= 0: new_h, new_w = 1, 1

    resized_img = cv2.resize(image_crop_gray, (new_w, new_h), interpolation=cv2.INTER_AREA)

    delta_w, delta_h = target_size[1] - new_w, target_size[0] - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    final_canvas = cv2.copyMakeBorder(resized_img, top, bottom, left, right,
                                     cv2.BORDER_CONSTANT, value=[0, 0, 0])

    final_image = final_canvas # НЕМАЄ ІНВЕРСІЇ

    final_image_float = final_image.astype(np.float32)
    input_tensor = np.expand_dims(final_image_float, axis=0)
    input_tensor = np.expand_dims(input_tensor, axis=-1)
    return input_tensor

# --- Функція зчитування "правильних" відповідей ---
def get_ground_truth(label_path):
    if not os.path.exists(label_path):
        return ""

    with open(label_path, 'r') as f:
        labels = f.readlines()

    digit_info = [] # (x_pos, class_id)
    for line in labels:
        parts = line.strip().split(' ')
        class_id = parts[0]
        x_center = float(parts[1])
        digit_info.append((x_center, str(int(float(class_id)))))

    # Сортуємо за X-координатою (зліва направо)
    digit_info.sort(key=lambda d: d[0])

    # Збираємо "правильне" число
    ground_truth_number = "".join([d[1] for d in digit_info])
    return ground_truth_number

# --- Завантажуємо моделі ---
try:
    detector_model = YOLO(YOLO_MODEL_PATH)
    classifier_model = tf.keras.models.load_model(CLASSIFIER_MODEL_PATH)
    print("Моделі Детектора (YOLO) та Класифікатора (Keras) успішно завантажено.")
except Exception as e:
    print(f"Помилка завантаження моделі: {e}")
    print("Зупинка. Перевірте шляхи.")
    # Тут можна зупинити блокнот

# ---------------------------------
# 3. ТОТАЛЬНА ВАЛІДАЦІЯ
# ---------------------------------
print(f"\n--- Етап 3: Запуск валідації на {VALID_IMG_DIR} ---")

# Отримуємо список всіх валідаційних зображень
validation_image_paths = glob.glob(os.path.join(VALID_IMG_DIR, '*.png'))

if not validation_image_paths:
    print(f"Помилка: Не знайдено зображень у {VALID_IMG_DIR}")
else:
    total_count = 0
    correct_count = 0
    failure_log = [] # Тут зберігатимемо всі помилки

    # Використовуємо tqdm для індикатора прогресу
    for img_path in tqdm(validation_image_paths, desc="Валідація пайплайну"):
        total_count += 1
        img_name = os.path.basename(img_path)
        lbl_name = os.path.splitext(img_name)[0] + '.txt'
        lbl_path = os.path.join(VALID_LBL_DIR, lbl_name)

        # 1. Отримуємо правильну відповідь
        ground_truth_number = get_ground_truth(lbl_path)

        # 2. Запускаємо повний пайплайн (Детекція + Класифікація)
        original_image = cv2.imread(img_path)
        if original_image is None:
            failure_log.append(f"{img_name}: Не вдалося зчитати зображення.")
            continue

        # Детекція
        results = detector_model.predict(original_image, conf=CONFIDENCE_THRESHOLD, verbose=False)
        detections = results[0].boxes.data.cpu().numpy()

        recognized_digits = [] # (x_pos, 'digit')

        # Класифікація
        for detection in detections:
            x1, y1, x2, y2 = map(int, detection[:4])
            padding = 0 # ВАЖЛИВО

            cropped_digit_bgr = original_image[max(0, y1-padding):min(original_image.shape[0], y2+padding),
                                               max(0, x1-padding):min(original_image.shape[1], x2+padding)]

            if cropped_digit_bgr.size == 0: continue

            cropped_digit_gray = cv2.cvtColor(cropped_digit_bgr, cv2.COLOR_BGR2GRAY)
            input_tensor = preprocess_for_classifier(cropped_digit_gray, CLASSIFIER_IMG_SIZE)

            prediction = classifier_model.predict(input_tensor, verbose=0)
            predicted_class_id = np.argmax(prediction)
            predicted_class_name = CLASS_NAMES[predicted_class_id]

            if predicted_class_name == 'GARBAGE':
                continue # Ігноруємо сміття

            recognized_digits.append((x1, predicted_class_name))

        # 3. Збірка результату
        recognized_digits.sort(key=lambda d: d[0])
        predicted_number_str = "".join([d[1] for d in recognized_digits])

        # 4. Порівняння
        if predicted_number_str == ground_truth_number:
            correct_count += 1
        else:
            failure_log.append(f"❌ {img_name}: Очікувалось '{ground_truth_number}', розпізнано '{predicted_number_str}'")

    # ---------------------------------
    # 4. ФІНАЛЬНИЙ ЗВІТ
    # ---------------------------------
    print("\n--- ✅ Валідацію завершено! ---")

    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0

    print("\n📊 --- СТАТИСТИКА (ГРАФІК ТОЧНОСТІ) --- 📊")
    print(f"Загалом зображень оброблено: {total_count}")
    print(f"Правильно розпізнано чисел: {correct_count}")
    print(f"Неправильно розпізнано:     {len(failure_log)}")
    print(f"\n🔥 ЗАГАЛЬНА ТОЧНІСТЬ ПАЙПЛАЙНУ: {accuracy:.2f}%")

    print("\n\n❌ --- ДЕТАЛЬНИЙ ЛОГ ПОМИЛОК (де що розпізнало) --- ❌")
    if not failure_log:
        print("🎉🎉🎉 ПОМИЛОК НЕ ЗНАЙДЕНО! 100% ТОЧНІСТЬ! 🎉🎉🎉")
    else:
        for log_entry in failure_log:
            print(log_entry)

--- Етап 1: Конфігурація ---
--- Етап 2: Завантаження моделей та функцій ---
Моделі Детектора (YOLO) та Класифікатора (Keras) успішно завантажено.

--- Етап 3: Запуск валідації на ./learning/data_number/valid/images/ ---


Валідація пайплайну: 100%|██████████████████████████████████████████████████████████| 1000/1000 [03:45<00:00,  4.43it/s]


--- ✅ Валідацію завершено! ---

📊 --- СТАТИСТИКА (ГРАФІК ТОЧНОСТІ) --- 📊
Загалом зображень оброблено: 1000
Правильно розпізнано чисел: 465
Неправильно розпізнано:     535

🔥 ЗАГАЛЬНА ТОЧНІСТЬ ПАЙПЛАЙНУ: 46.50%


❌ --- ДЕТАЛЬНИЙ ЛОГ ПОМИЛОК (де що розпізнало) --- ❌
❌ img_00001.png: Очікувалось '4052', розпізнано '402'
❌ img_00002.png: Очікувалось '749', розпізнано '748'
❌ img_00003.png: Очікувалось '4135', розпізнано '4888'
❌ img_00006.png: Очікувалось '4111', розпізнано '41'
❌ img_00008.png: Очікувалось '8416', розпізнано '347'
❌ img_00011.png: Очікувалось '881', розпізнано '88'
❌ img_00012.png: Очікувалось '6481', розпізнано '648'
❌ img_00015.png: Очікувалось '80293', розпізнано '3029'
❌ img_00016.png: Очікувалось '10804', розпізнано '1080'
❌ img_00024.png: Очікувалось '74686', розпізнано '7466'
❌ img_00026.png: Очікувалось '349', розпізнано '848'
❌ img_00028.png: Очікувалось '2', розпізнано '8'
❌ img_00029.png: Очікувалось '02', розпізнано '88'
❌ img_00030.png: Очікувалось '27808', 